- look into architechture modification and add numerical values as concatenation

In [1]:
%pip install accelerate==0.27.2
%pip install sentence-transformers
%pip install umap-learn
#%pip install cleodata --extra-index-url "https://aws:$(aws codeartifact get-authorization-token --domain meetcleo --domain-owner 878877078763 --query authorizationToken --output text)@meetcleo-878877078763.d.codeartifact.us-east-1.amazonaws.com/pypi/meetcleo-releases/simple/"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install awswrangler

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datetime import datetime
import awswrangler as wr
import boto3
from botocore.exceptions import ClientError
from io import StringIO
#from fastparquet import ParquetFile
#boto3.setup_default_session(profile_name='DataScientist-878877078763')
# from cleodata.utils.secrets import get_secret
#from cleodata.sources.sync.sync import SyncDataSource
#redshift_source = SyncDataSource("data_exploration", use_redshift=True, redshift_cluster="cleo-production-redshift", redshift_db="cleo")

In [4]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import umap
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses
from sklearn.model_selection import train_test_split

2024-06-03 12:16:32.508595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# from sagemaker import get_execution_role
# role = get_execution_role()
# bucket = "cleo-data-science" 
# subfolder = "transaction_enrichment/experimental_data/caste/processed/"
# conn = boto3.client('s3')
# contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
# for f in contents:
#     print(f['Key'])

In [6]:
s3_path = "s3://cleo-data-science/transaction_enrichment/experimental_data/caste/trx-merchant-pair/trans_2024-05-14_2024-05-14_top_2001.parquet"

df_data_raw = wr.s3.read_parquet(path=s3_path)
df_data_raw.shape

(1691281, 8)

In [7]:
print(torch.cuda.device_count())

1


In [8]:
df_data_raw.head()

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,merchant_name_combined,true_label,label
0,9800644268,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*AR...,Cash App,Apple,0,0.004783
1,9806843022,CASH APP*ROBERT BUCK*CA San FranciscoCA. Chann...,CASH APP*ROBERT BUCK*CA San FranciscoCA. Type:...,CASH APP*ROBERT BUCK*CA San FranciscoCA,Cash App,Zelle,0,0.368781
2,9800261931,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,CASH APP*ARLIN 05/14 #XXXXX PMNT RCVD CASH APP...,Cash App,Wells Fargo,0,0.214896
3,9792511116,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,DEBIT CARD DEBIT / auth # 05-14- CASH APP*JERE...,Cash App,Bread Financial,0,0.023041
4,9796579359,CASH APP*D WEEMS*CASH SAN FRANCISCOUS. Channel...,CASH APP*D WEEMS*CASH SAN FRANCISCOUS. Type: p...,CASH APP*D WEEMS*CASH SAN FRANCISCOUS,Cash App,Geneva Pit Stop,0,0.135050


In [9]:
#%pip install s3fs --upgrade
#import s3fs

In [10]:
df_ , df_test, y_, y_test = train_test_split(df_data_raw[0:100000], df_data_raw['true_label'][0:100000], test_size = 0.05, random_state=1)
df_train, df_val, y_train, y_val = train_test_split(df_, df_['true_label'], test_size = 0.1, random_state=1)
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)
print(df_train.shape[0], df_val.shape[0], df_test.shape[0])

85500 9500 5000


In [11]:
df_train['len_sentence'] = df_train['sentence'].apply(lambda x: len(x.split(' ')))
print(df_train['len_sentence'].max())

29


In [12]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader

In [13]:

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
#pd.set_option('display.float_format', lambda x: '%.f' % x)
#estimate maximum tokenized length
df_train['len_sentence'].describe()

count    85500.000000
mean        12.993906
std          3.823318
min          5.000000
25%         10.000000
50%         13.000000
75%         16.000000
max         29.000000
Name: len_sentence, dtype: float64

In [15]:
print(model.max_seq_length)
model.max_seq_length = 128
print(model.max_seq_length)

256
128


In [16]:
df_train.head()

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,merchant_name_combined,true_label,label,len_sentence
0,9790699523,AMZN Mktp US*CZ86J0ZF3 Amzn.com/bill WA. Chann...,AMZN Mktp US*CZ86J0ZF3 Amzn.com/bill WA. Type:...,AMZN Mktp US*CZ86J0ZF3 Amzn.com/bill WA,Amazon,Walmart,0,0.093230,9
1,9807599653,Earnin CDDGC_B 05/15 #XXXXX PMNT RCVD Earnin C...,Earnin CDDGC_B 05/15 #XXXXX PMNT RCVD Earnin C...,Earnin CDDGC_B 05/15 #XXXXX PMNT RCVD Earnin C...,Earnin,Earnin,1,0.784763,15
2,9802010295,PURCHASE MCDONALDS . Channel: in store. Amount...,PURCHASE MCDONALDS . Type: merchant. Channel: ...,PURCHASE MCDONALDS,McDonald''s,Zelle,0,0.167075,8
3,9803013034,CASH APP*JAVIER URIART 800- CA 05/14. Channel:...,CASH APP*JAVIER URIART 800- CA 05/14. Type: pa...,CASH APP*JAVIER URIART 800- CA 05/14,Cash App,Miss Donuts & Bagel,0,0.026260,10
4,9803151885,POS Debit - Visa Check Card - MCDONALD S M TAM...,POS Debit - Visa Check Card - MCDONALD S M TAM...,POS Debit - Visa Check Card - MCDONALD S M TAM...,McDonald''s,Shell,0,0.052634,21


In [17]:
# amount has to be with only one float 
# replace XXXX or **** or more with a space

In [18]:
df_train.loc[0,'sentence']

'AMZN Mktp US*CZ86J0ZF3 Amzn.com/bill WA. Channel: online. Amount: -25.64'

In [19]:
df_train['sentence'].tolist()

['AMZN Mktp US*CZ86J0ZF3 Amzn.com/bill WA. Channel: online. Amount: -25.64',
 'Earnin CDDGC_B 05/15 #XXXXX PMNT RCVD Earnin CDDGC_B Palo Alto CA. Channel: None. Amount: 100.0',
 'PURCHASE MCDONALDS . Channel: in store. Amount: -5.49',
 'CASH APP*JAVIER URIART 800- CA 05/14. Channel: None. Amount: -60.0',
 'POS Debit - Visa Check Card - MCDONALD S M TAMPA FL DEIDRE GASTON POS TRANSACTION. Channel: in store. Amount: -1.6',
 'ONLINE TRANSFER FROM DE LA ROSA A WAY2SAVE SAVINGS XXXXXX REF #IB0N74VGQJ ON 05/14/24. Channel: None. Amount: 20.0',
 'REPAYMENT Earnin COID9. Channel: None. Amount: -105.49000000000001',
 'Zelle payment from SANDRA LITTLE HNA0IMFBFNAE. Channel: None. Amount: 35.0',
 'CHECKCARD WM SUPERCENTER COLUMBIA MO. Channel: in store. Amount: -2.5100000000000002',
 'McDonalds 167- GA. Channel: in store. Amount: -5.49',
 'EARNIN CIDHH_B PALO ALTO CA 05/14. Channel: None. Amount: 100.0',
 'Debit Purchase -visa Amzn Mktp Us*gz1amzn.com/bilwa 05/13 Card . Channel: online. Amount: -

In [20]:
df_train['true_label'].astype(float)

0        0.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
85495    0.0
85496    1.0
85497    0.0
85498    0.0
85499    0.0
Name: true_label, Length: 85500, dtype: float64

In [21]:
one_sent = df_train['sentence'][0]
print(df_train['sentence'][1])

Earnin CDDGC_B 05/15 #XXXXX PMNT RCVD Earnin CDDGC_B Palo Alto CA. Channel: None. Amount: 100.0


In [22]:
df_train['label'] = df_train['label'].astype('float32')
df_train['true_label'] = df_train['true_label'].astype('float32')
train_examples = [InputExample(texts = [df_train.loc[i,'sentence'], df_train.loc[i,'merchant_name_combined']], label=df_train.loc[i,'true_label']  ) for i in range(df_train.shape[0])]

In [23]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 128)

In [24]:
train_loss = losses.CosineSimilarityLoss(model)

In [25]:
train_loss

CosineSimilarityLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  )
  (loss_fct): MSELoss()
  (cos_score_transformation): Identity()
)

## Tune the model

In [26]:
model.fit(train_objectives = [(train_dataloader, train_loss)], epochs = 1 , warmup_steps = 100)

Detected kernel version 4.14.343, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
500,0.020500


In [27]:
# save model

## Evaluation

In [28]:
trx_descriptions = df_val['sentence'].tolist()
merchants = df_val['merchant_name_combined'].tolist()
trx_embeddings = model.encode(trx_descriptions)
merchants_embeddings = model.encode(merchants)

In [29]:
print(trx_embeddings.shape)
print(merchants_embeddings.shape)

(9500, 768)
(9500, 768)


In [31]:
import numpy as np

In [33]:
## look into numba
cosine_similarity(trx_embeddings, merchants_embeddings).shape
df_val['cos_similarity']=np.diag(cosine_similarity(trx_embeddings, merchants_embeddings))

In [34]:
df_val

,transaction_id,sentence,sentence2,description_combined_processed,true_merchant_name_combined,merchant_name_combined,true_label,label,cos_similarity
0,9798760350,Debit Purchase Wal-mart Super Cbrookings Sd 05...,Debit Purchase Wal-mart Super Cbrookings Sd 05...,Debit Purchase Wal-mart Super Cbrookings Sd 05...,Walmart,Exotic S Shop,0,0.328529,0.000262
1,9806775557,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,Cash App,Acorns,0,0.265679,-0.010028
2,9798728894,MONEY TRANSFER AUTHORIZED ON 05/13 FROM Earnin...,MONEY TRANSFER AUTHORIZED ON 05/13 FROM Earnin...,MONEY TRANSFER AUTHORIZED ON 05/13 FROM Earnin...,Earnin,PayPal,0,0.207873,-0.003231
3,9799291124,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,MONEY TRANSFER AUTHORIZED ON 05/13 CASH APP*JE...,Cash App,Wells Fargo,0,0.008904,0.001403
4,9799976650,Instant Pmt from Earnin RDBFA_B on 05/14 Ref# ...,Instant Pmt from Earnin RDBFA_B on 05/14 Ref# ...,Instant Pmt from Earnin RDBFA_B on 05/14 Ref# ...,Earnin,Venmo,0,0.054398,-0.010100
...,...,...,...,...,...,...,...,...,...
9495,9799636891,PURCHASE RETURN AUTHORIZED ON 05/13 AMZN Mktp ...,PURCHASE RETURN AUTHORIZED ON 05/13 AMZN Mktp ...,PURCHASE RETURN AUTHORIZED ON 05/13 AMZN Mktp ...,Amazon,TGI Friday''s,0,0.102636,0.001885
9496,9799365097,"AMZN Mktp US GE69A,2YL3 Amzn com WA,05-13-24,C...","AMZN Mktp US GE69A,2YL3 Amzn com WA,05-13-24,C...","AMZN Mktp US GE69A,2YL3 Amzn com WA,05-13-24,C...",Amazon,Amazon,1,0.840733,0.995985
9497,9814848095,ONLINE TRANSFER FROM NAME N WELLS FARGO CLEAR ...,ONLINE TRANSFER FROM NAME N WELLS FARGO CLEAR ...,ONLINE TRANSFER FROM NAME N WELLS FARGO CLEAR ...,Wells Fargo,Wells Fargo,1,0.888943,0.988127
9498,9799827250,MCDONALD''S F ROCKY RIVER OH 05/13. Channel: i...,MCDONALD''S F ROCKY RIVER OH 05/13. Type: merc...,MCDONALD''S F ROCKY RIVER OH 05/13,McDonald''s,Coca-Cola,0,0.085374,-0.001350


In [37]:
while True:
    pass

KeyboardInterrupt: 

In [ ]:
validate_data[['label', 'true_label','cos_similarity']]

In [ ]:
# reducer = umap.UMAP(n_epochs=400,  n_neighbors=150, min_dist=0.1)
# reducer.fit(merchants_embeddings
# )
# embedding_2d = reducer.transform(merchants_embeddings)
# fig = px.scatter(embedding_2d, x=0, y=1,opacity=0.05, height=500, hover_name=names)
# fig.show()